In [33]:
import pandas as pd 
import numpy as np
import pyblp
from sklearn.linear_model import LinearRegression

In [34]:
product_data = pd.read_csv('../data/market_integrates_1.csv')
print(product_data)

     market_ids  firm_ids  characteristic1  characteristic2    prices  \
0             1         1         3.988608         3.958960  2.683035   
1             1         2         5.319729         3.979387  2.327647   
2             1         3         4.342128         4.835318  2.324931   
3             1         4         5.405299         4.302028  2.269334   
4             1         5         3.586754         4.746072  2.517294   
..          ...       ...              ...              ...       ...   
995         100         6         3.103106         5.520905  2.556986   
996         100         7         5.806308         3.553468  2.403566   
997         100         8         4.888032         3.012261  2.593767   
998         100         9         4.496945         5.025356  2.239298   
999         100        10         3.499739         4.028382  2.644176   

     marginal_cost    shares   profits   markups  e_quantity    capital  \
0         1.896486  0.008741  0.006875  1.414741

In [35]:
# # Getting the dataset estimation ready with at least a supply_side instrument/ demand_side instrument 
product_data_complete = product_data.rename(columns={'labor':'supply_instrument0',
                                                      'capital':'supply_instruments1',
                                                        'marginal_cost': 'supply_instrument2'
                                                        })



In [36]:
# the endog regressor
X1_formulation = pyblp.Formulation('1  + prices+ characteristic1 + characteristic2')

# the only regressor on which we want random coeff 
X2_formulation = pyblp.Formulation('0 +  prices')

# supply side charact 
# X3_formulation = pyblp.Formulation('1 + log(labor) + log(capital)')

# formulation for the cost thing (assume cost fomrulation being labor and capital)
product_formulations = (X1_formulation,
                         X2_formulation
                           #  X3_formulation
                           )
print(product_formulations)

(1 + prices + characteristic1 + characteristic2, prices)


In [37]:
mc_integration = pyblp.Integration('monte_carlo', size=1000, specification_options={'seed': 0})


In [38]:
mc_problem = pyblp.Problem(product_formulations, product_data_complete, integration=mc_integration)
print(mc_problem)

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     F     I      K1    K2    MD 
---  ----  ---  ------  ----  ----  ----
100  1000  10   100000   4     1     3  

Formulations:
       Column Indices:           0       1            2                3       
-----------------------------  ------  ------  ---------------  ---------------
 X1: Linear Characteristics      1     prices  characteristic1  characteristic2
X2: Nonlinear Characteristics  prices                                          
Dimensions:
 T    N     F     I      K1    K2    MD 
---  ----  ---  ------  ----  ----  ----
100  1000  10   100000   4     1     3  

Formulations:
       Column Indices:           0       1            2                3       
-----------------------------  ------  ------  ---------------  ---------------
 X1: Linear Characteristics      1     prices  characteristic1  characteristic2
X2: Nonlinear Characteristics  prices                                 

In [39]:
bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-4})


In [40]:
results1 = mc_problem.solve(sigma=np.ones((1, 1)), optimization=bfgs)


Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:     prices    
------  -------------
prices  +1.000000E+00
Starting optimization ...

GMM   Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Gradient                  
Step   Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement       Norm           Theta    
----  ------------  -----------  -----------  -----------  -------  -------------  -------------  -------------  -------------
 1         0             1           505         1612         0     +2.962950E-23                 +6.047646E-10  +1.000000E+00

Optimization completed after 00:00:01.
Computing the Hessian and and updating the weighting matrix ...
Computed results after 00:00:02.

Problem Results Summary:
GMM     Objective      Gradient                    Clipped  Weighting Matrix
Step      Value          Norm          Hessian     Shares   Condition Number
----  -------------